# 1. Prepration

## 1.1 Retrieve dataset from github

In [2]:
!git clone https://github.com/indobenchmark/indonlu

fatal: destination path 'indonlu' already exists and is not an empty directory.


## 1.2 Import Library

In [3]:
import random
import numpy as np
import pandas as pd
import torch
from torch import optim
import torch.nn.functional as F
from tqdm import tqdm
 
from transformers import BertForSequenceClassification, BertConfig, BertTokenizer
from nltk.tokenize import TweetTokenizer
 
from indonlu.utils.forward_fn import forward_sequence_classification
from indonlu.utils.metrics import document_sentiment_metrics_fn
from indonlu.utils.data_utils import DocumentSentimentDataset, DocumentSentimentDataLoader

In [4]:
# np.set_printoptions(linewidth=np.inf)

## 1.3 Define Function

In [5]:
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    
def count_param(module, trainable=False):
    if trainable:
        return sum(p.numel() for p in module.parameters() if p.requires_grad)
    else:
        return sum(p.numel() for p in module.parameters())
    
def get_lr(optimizer):
    for param_group in optimizer.param_groups:
        return param_group['lr']
 
def metrics_to_string(metric_dict):
    string_list = []
    for key, value in metric_dict.items():
        string_list.append('{}:{:.2f}'.format(key, value))
    return ' '.join(string_list)

In [6]:
# Set random seed
set_seed(19072021)

## 1.4 Setup Pre-trained Model

In [7]:
# Load Tokenizer and Config
tokenizer = BertTokenizer.from_pretrained('indobenchmark/indobert-base-p1')
config = BertConfig.from_pretrained('indobenchmark/indobert-base-p1')
config.num_labels = DocumentSentimentDataset.NUM_LABELS
 
# Instantiate model
model = BertForSequenceClassification.from_pretrained('indobenchmark/indobert-base-p1', config=config)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at indobenchmark/indobert-base-p1 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [8]:
count_param(model)

124443651

## 1.5 Setup Dataset Path

In [9]:
train_dataset_path = "/home/yelf/Desktop/dbs_ML_expert/#02-SentimentAnalysis/indonlu/dataset/smsa_doc-sentiment-prosa/train_preprocess.tsv"
valid_dataset_path = "/home/yelf/Desktop/dbs_ML_expert/#02-SentimentAnalysis/indonlu/dataset/smsa_doc-sentiment-prosa/valid_preprocess.tsv"
test_dataset_path = "/home/yelf/Desktop/dbs_ML_expert/#02-SentimentAnalysis/indonlu/dataset/smsa_doc-sentiment-prosa/test_preprocess.tsv"

## 1.6 Define Dataset and Loader

In [10]:
from torch.utils.data import Dataset, DataLoader

In [11]:
# # Kode berikut hanya untuk menunjukan bagaimana kelas DocumenSentiment Dataset dan Loader bekerja / Tidak perlu ditulis
# class DocumentSentimentDataset(Dataset):
#     # Static constant variable
#     LABEL2INDEX = {'positive': 0, 'neutral': 1, 'negative': 2} # Map dari label string ke index
#     INDEX2LABEL = {0: 'positive', 1: 'neutral', 2: 'negative'} # Map dari Index ke label string
#     NUM_LABELS = 3 # Jumlah label
   
#     def load_dataset(self, path):
#         df = pd.read_csv(path, sep='\t', header=None) # Baca tsv file dengan pandas
#         df.columns = ['text','sentiment'] # Berikan nama pada kolom tabel
#         df['sentiment'] = df['sentiment'].apply(lambda lab: self.LABEL2INDEX[lab]) # Konversi string label ke index
#         return df
   
#     def __init__(self, dataset_path, tokenizer, *args, **kwargs):
#         self.data = self.load_dataset(dataset_path) # Load tsv file
 
#         # Assign tokenizer, disini kita menggunakan tokenizer subword dari HuggingFace
#         self.tokenizer = tokenizer 
 
#     def __getitem__(self, index):
#         data = self.data.loc[index,:] # Ambil data pada baris tertentu dari tabel
#         text, sentiment = data['text'], data['sentiment'] # Ambil nilai text dan sentiment
#         subwords = self.tokenizer.encode(text) # Tokenisasi text menjadi subword
    
#         # Return numpy array dari subwords dan label
#         return np.array(subwords), np.array(sentiment), data['text']
   
#     def __len__(self):
#         return len(self.data)  # Return panjang dari dataset

# class DocumentSentimentDataLoader(DataLoader):
#     def __init__(self, max_seq_len=512, *args, **kwargs):
#         super(DocumentSentimentDataLoader, self).__init__(*args, **kwargs)
#         self.max_seq_len = max_seq_len # Assign batas maksimum subword
#         self.collate_fn = self._collate_fn # Assign fungsi collate_fn dengan fungsi yang kita definisikan
       
#     def _collate_fn(self, batch):
#         batch_size = len(batch) # Ambil batch size
#         max_seq_len = max(map(lambda x: len(x[0]), batch)) # Cari panjang subword maksimal dari batch 
#         max_seq_len = min(self.max_seq_len, max_seq_len) # Bandingkan dengan batas yang kita tentukan sebelumnya
       
#         # Buat buffer untuk subword, mask, dan sentiment labels, inisialisasikan semuanya dengan 0
#         subword_batch = np.zeros((batch_size, max_seq_len), dtype=np.int64)
#         mask_batch = np.zeros((batch_size, max_seq_len), dtype=np.float32)
#         sentiment_batch = np.zeros((batch_size, 1), dtype=np.int64)
       
#         # Isi semua buffer
#         for i, (subwords, sentiment, raw_seq) in enumerate(batch):
#             subwords = subwords[:max_seq_len]
#             subword_batch[i,:len(subwords)] = subwords
#             mask_batch[i,:len(subwords)] = 1
#             sentiment_batch[i,0] = sentiment
           
#         # Return subword, mask, dan sentiment data
#         return subword_batch, mask_batch, sentiment_batch

In [12]:
train_dataset = DocumentSentimentDataset(train_dataset_path, tokenizer, lowercase=True)
valid_dataset = DocumentSentimentDataset(valid_dataset_path, tokenizer, lowercase=True)
test_dataset = DocumentSentimentDataset(test_dataset_path, tokenizer, lowercase=True)
 
train_loader = DocumentSentimentDataLoader(dataset=train_dataset, max_seq_len=512, batch_size=32, num_workers=12, shuffle=True)  
valid_loader = DocumentSentimentDataLoader(dataset=valid_dataset, max_seq_len=512, batch_size=32, num_workers=12, shuffle=False)  
test_loader = DocumentSentimentDataLoader(dataset=test_dataset, max_seq_len=512, batch_size=32, num_workers=12, shuffle=False)

In [13]:
print(train_dataset[0])

(array([    2,  6540,    92,  2970,   213,  4259,  3553,   899,    34,
         259,  5590,   262,  2558,   386,   899,  1687,    26,  1574,
       30470,   899,  3310, 30468, 22130, 30360,  6123,  6368, 30468,
       22130, 30360,  2652,  1746, 30468,  8869,  6540,    34,  6315,
        1622,  1256,  8949,   899, 30468,  4222,  1622,   752,   245,
         295,  2083, 30470,  2346,  7107,   300, 30470,   405,   724,
        5189, 30470,   843, 17464,   899,   540, 10989,  3331,  1107,
       30468,   119,  3221,    79,    34,  2170,    98,  9167, 30457,
           3]), array(0), 'warung ini dimiliki oleh pengusaha pabrik tahu yang sudah puluhan tahun terkenal membuat tahu putih di bandung . tahu berkualitas , dipadu keahlian memasak , dipadu kretivitas , jadilah warung yang menyajikan menu utama berbahan tahu , ditambah menu umum lain seperti ayam . semuanya selera indonesia . harga cukup terjangkau . jangan lewatkan tahu bletoka nya , tidak kalah dengan yang asli dari tegal !')


In [14]:
w2i = DocumentSentimentDataset.LABEL2INDEX  # word to index
i2w = DocumentSentimentDataset.INDEX2LABEL  # index to word
print("Word to Index :", w2i)
print("Index to Word :", i2w)

Word to Index : {'positive': 0, 'neutral': 1, 'negative': 2}
Index to Word : {0: 'positive', 1: 'neutral', 2: 'negative'}


# 2. Uji Model

In [15]:
text = 'Bahagia hatiku melihat pernikahan putri sulungku yang cantik jelita'
print("Text     :", text)

subwords = tokenizer.encode(text)
print("Token    :", subwords)
subwords = torch.LongTensor(subwords).view(1, -1).to(model.device)  # convert to torch tensor
print("Token(2) :", subwords)

logits = model(subwords)[0]
print("logits   :", logits)

label = torch.topk(logits, k=1, dim=-1)[1].squeeze().item()
print("Label    :", label)

Text     : Bahagia hatiku melihat pernikahan putri sulungku yang cantik jelita
Token    : [2, 4771, 10413, 722, 3300, 3466, 19227, 457, 34, 2176, 17377, 155, 3]
Token(2) : tensor([[    2,  4771, 10413,   722,  3300,  3466, 19227,   457,    34,  2176,
         17377,   155,     3]])
logits   : tensor([[ 0.5018, -0.2294, -0.3558]], grad_fn=<AddmmBackward0>)
Label    : 0


In [16]:
print(f'Text: {text} | Label : {i2w[label]} ({F.softmax(logits, dim=-1).squeeze()[label] * 100:.3f}%)')

Text: Bahagia hatiku melihat pernikahan putri sulungku yang cantik jelita | Label : positive (52.480%)


# 3. Fine Tuning dan Evaluasi

## 3.1 Fine Tuning

In [17]:
# set optimezier to Adam
optimizer = optim.Adam(model.parameters(), lr=3e-6) # learning rate = 0.000003
model = model.cuda()

In [ ]:
import os

# os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "max_split_size_mb:512"
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

In [ ]:
# Train
n_epochs = 5
for epoch in range(n_epochs):
    model.train()
    torch.set_grad_enabled(True)
 
    total_train_loss = 0
    list_hyp, list_label = [], []
 
    train_pbar = tqdm(train_loader, leave=True, total=len(train_loader))
    for i, batch_data in enumerate(train_pbar):
        # Forward model
        loss, batch_hyp, batch_label = forward_sequence_classification(model, batch_data[:-1], i2w=i2w, device='cuda')
 
        # Update model
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
 
        tr_loss = loss.item()
        total_train_loss = total_train_loss + tr_loss
 
        # Calculate metrics
        list_hyp += batch_hyp
        list_label += batch_label
 
        train_pbar.set_description("(Epoch {}) TRAIN LOSS:{:.4f} LR:{:.8f}".format((epoch+1),
            total_train_loss/(i+1), get_lr(optimizer)))
 
    # Calculate train metric
    metrics = document_sentiment_metrics_fn(list_hyp, list_label)
    print("(Epoch {}) TRAIN LOSS:{:.4f} {} LR:{:.8f}".format((epoch+1),
        total_train_loss/(i+1), metrics_to_string(metrics), get_lr(optimizer)))
 
    # Evaluate on validation
    model.eval()
    torch.set_grad_enabled(False)
    
    total_loss, total_correct, total_labels = 0, 0, 0
    list_hyp, list_label = [], []
 
    pbar = tqdm(valid_loader, leave=True, total=len(valid_loader))
    for i, batch_data in enumerate(pbar):
        batch_seq = batch_data[-1]        
        loss, batch_hyp, batch_label = forward_sequence_classification(model, batch_data[:-1], i2w=i2w, device='cuda')
        
        # Calculate total loss
        valid_loss = loss.item()
        total_loss = total_loss + valid_loss
 
        # Calculate evaluation metrics
        list_hyp += batch_hyp
        list_label += batch_label
        metrics = document_sentiment_metrics_fn(list_hyp, list_label)
 
        pbar.set_description("VALID LOSS:{:.4f} {}".format(total_loss/(i+1), metrics_to_string(metrics)))
        
    metrics = document_sentiment_metrics_fn(list_hyp, list_label)
    print("(Epoch {}) VALID LOSS:{:.4f} {}".format((epoch+1),
        total_loss/(i+1), metrics_to_string(metrics)))